Data comes from https://github.com/AMLab-Amsterdam/CEVAE/tree/master/datasets/TWINS

In [39]:

import pandas as pd
import json
from tqdm import tqdm

In [24]:
df_t = pd.read_csv("twin_pairs_T_3years_samesex.csv")
df_c_ = pd.read_csv("twin_pairs_X_3years_samesex.csv")
df_o = pd.read_csv("twin_pairs_Y_3years_samesex.csv")

In [26]:
df_c = df_c_[df_c_.columns[2:]].drop(columns=["infant_id_0", "infant_id_1"]).dropna()
df_c

,pldel,birattnd,brstate,stoccfipb,mager8,ormoth,mrace,meduc6,dmar,mplbir,...,nprevistq,dfageq,feduc6,dlivord_min,dtotord_min,bord_0,bord_1,brstate_reg,stoccfipb_reg,mplbir_reg
3,1.0,1.0,1,1,4,0.0,2,1.0,1,1.0,...,1.0,6.0,1.0,4.0,4.0,1.0,2.0,5,5,5.0
6,1.0,1.0,1,1,3,0.0,1,3.0,1,1.0,...,3.0,0.0,3.0,1.0,2.0,1.0,2.0,5,5,5.0
8,1.0,1.0,1,1,4,0.0,1,4.0,1,28.0,...,3.0,3.0,4.0,1.0,1.0,1.0,2.0,5,5,3.0
9,1.0,1.0,1,1,3,0.0,1,3.0,1,1.0,...,0.0,1.0,3.0,3.0,3.0,2.0,1.0,5,5,5.0
13,1.0,1.0,1,1,5,0.0,1,4.0,1,1.0,...,4.0,6.0,4.0,1.0,1.0,1.0,2.0,5,5,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71339,1.0,1.0,51,56,2,0.0,1,2.0,1,51.0,...,3.0,0.0,4.0,2.0,2.0,2.0,1.0,7,7,7.0
71340,1.0,1.0,51,56,4,0.0,3,4.0,0,51.0,...,1.0,0.0,3.0,5.0,5.0,2.0,1.0,7,7,7.0
71341,1.0,1.0,51,56,4,0.0,3,3.0,0,51.0,...,0.0,5.0,2.0,6.0,7.0,2.0,1.0,7,7,7.0
71342,1.0,1.0,51,56,2,0.0,1,2.0,1,38.0,...,0.0,0.0,2.0,1.0,1.0,1.0,2.0,7,7,8.0


In [35]:
shared_covs = [c for c in df_c.columns if not "_0" in c and not "_1" in c]
individual_covs = sorted(set([c[:-2] for c in df_c.columns if c.endswith(("_0", "_1"))]))

In [ ]:
rows = []
pbar = tqdm(total=len(df_c))
for i, row in df_c.iterrows():
    mort_0, mort_1 = df_o.loc[i][["mort_0", "mort_1"]]
    intervention_dict = [
        {
            "t": {"heavier": 0},
            "o": {"mort": mort_0}
        },
        {
            "t": {"heavier": 1},
            "o": {"mort": mort_1}
        }
    ]
    rows.append([row[c] for c in shared_covs] + [intervention_dict])
    pbar.update(1)
    break
pbar.close()
df_compiled = pd.DataFrame(rows, columns=[f"c:{c}" for c in shared_covs] + ["outcomes"])

  0%|          | 1/32120 [00:00<02:37, 203.40it/s]


ValueError: 49 columns passed, passed data had 50 columns

In [45]:
display(df_compiled)
df_compiled.to_csv("twins.csv")

,c:pldel,c:birattnd,c:brstate,c:stoccfipb,c:mager8,c:ormoth,c:mrace,c:meduc6,c:dmar,c:mplbir,...,c:nprevistq,c:dfageq,c:feduc6,c:dlivord_min,c:dtotord_min,c:brstate_reg,c:stoccfipb_reg,c:mplbir_reg,o:mort(0),o:mort(1)
0,1.0,1.0,1.0,1.0,4.0,0.0,2.0,1.0,1.0,1.0,...,1.0,6.0,1.0,4.0,4.0,5.0,5.0,5.0,0.0,0.0
1,1.0,1.0,1.0,1.0,3.0,0.0,1.0,3.0,1.0,1.0,...,3.0,0.0,3.0,1.0,2.0,5.0,5.0,5.0,0.0,0.0
2,1.0,1.0,1.0,1.0,4.0,0.0,1.0,4.0,1.0,28.0,...,3.0,3.0,4.0,1.0,1.0,5.0,5.0,3.0,0.0,0.0
3,1.0,1.0,1.0,1.0,3.0,0.0,1.0,3.0,1.0,1.0,...,0.0,1.0,3.0,3.0,3.0,5.0,5.0,5.0,0.0,0.0
4,1.0,1.0,1.0,1.0,5.0,0.0,1.0,4.0,1.0,1.0,...,4.0,6.0,4.0,1.0,1.0,5.0,5.0,5.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32115,1.0,1.0,51.0,56.0,2.0,0.0,1.0,2.0,1.0,51.0,...,3.0,0.0,4.0,2.0,2.0,7.0,7.0,7.0,0.0,0.0
32116,1.0,1.0,51.0,56.0,4.0,0.0,3.0,4.0,0.0,51.0,...,1.0,0.0,3.0,5.0,5.0,7.0,7.0,7.0,0.0,0.0
32117,1.0,1.0,51.0,56.0,4.0,0.0,3.0,3.0,0.0,51.0,...,0.0,5.0,2.0,6.0,7.0,7.0,7.0,7.0,0.0,0.0
32118,1.0,1.0,51.0,56.0,2.0,0.0,1.0,2.0,1.0,38.0,...,0.0,0.0,2.0,1.0,1.0,7.0,7.0,8.0,0.0,0.0
